## Урок 3. Линейные преобразования

### Линейные преобразования

__1.__ Найти собственные векторы и собственные значения для линейного оператора, заданного матрицей

$$A=\begin{pmatrix}
-1 & -6\\ 
2 & 6
\end{pmatrix}.$$


__Решение__

$$\begin{vmatrix}
-1-\lambda & -6\\ 
2 & 6-\lambda
\end{vmatrix} = 0,$$

$$(-1-\lambda)(6-\lambda)-2\cdot(-6)=0,$$

$$\lambda^{2}-5\lambda+6=0,$$

Собственные значения: $$\lambda_{1}=2, \lambda_{2}=3.$$

$\textbf{A}x=\lambda x$:

$$\begin{pmatrix}
-1 & -6\\ 
2 & 6
\end{pmatrix}
\begin{pmatrix}
x_{1}\\ 
x_{2}
\end{pmatrix}=
\lambda
\begin{pmatrix}
x_{1}\\ 
x_{2}
\end{pmatrix}.
$$

Для собственного значения $\lambda_{1}=2:$

$$\begin{cases}
-1 \cdot x_{1}-6 \cdot x_{2}=2 \cdot x_{1}, \\
2 \cdot x_{1}+6 \cdot x_{2}=2 \cdot x_{2}.
\end{cases}$$ 

$$\begin{cases}
-1 \cdot x_{1}-6 \cdot x_{2}-2 \cdot x_{1}=0, \\
2 \cdot x_{1}+6 \cdot x_{2}-2 \cdot x_{2}=0.
\end{cases}$$ 

$$\begin{cases}
-3 \cdot x_{1}-6 \cdot x_{2}=0, \\
2 \cdot x_{1}+4 \cdot x_{2}=0.
\end{cases}$$ 

$$\begin{cases}
1 \cdot x_{1}+2 \cdot x_{2}=0, \\
1 \cdot x_{1}+2 \cdot x_{2}=0.
\end{cases}$$ 

$$\begin{cases}
x_{1}= -2 \cdot x_{2}, \\
x_{2} = -0.5 \cdot x_{1}.
\end{cases}$$

Собственный вектор 1 - это любой вектор с координатой $x_{1}$ в -2 раза большей чем координата $x_{2}.$ Например, вектор с координатами $x_{1}=-2$, $x_{2}=1.$

Для собственного значения $\lambda_{2}=3:$

$$\begin{cases}
-1 \cdot x_{1}-6 \cdot x_{2}=3 \cdot x_{1}, \\
2 \cdot x_{1}+6 \cdot x_{2}=3 \cdot x_{2}.
\end{cases}$$ 

$$\begin{cases}
-1 \cdot x_{1}-6 \cdot x_{2}-3 \cdot x_{1}=0, \\
2 \cdot x_{1}+6 \cdot x_{2}-3 \cdot x_{2}=0.
\end{cases}$$ 

$$\begin{cases}
-4 \cdot x_{1}-6 \cdot x_{2}=0, \\
2 \cdot x_{1}+3 \cdot x_{2}=0.
\end{cases}$$ 

$$\begin{cases}
1 \cdot x_{1}+1.5 \cdot x_{2}=0, \\
1 \cdot x_{1}+1.5 \cdot x_{2}=0.
\end{cases}$$ 

$$\begin{cases}
x_{1}= -1.5 \cdot x_{2}, \\
x_{2} = -\frac{2}{3} \cdot x_{1}.
\end{cases}$$

Собственный вектор 2 - это любой вектор с координатой $x_{1}$ в -1.5 раза большей чем координата $x_{2}.$ Например, вектор с координатами $x_{1}=3$, $x_{2}=-2.$

In [6]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [7]:
a = np.array([[-1, -6],
              [2, 6]])
w, v = np.linalg.eig(a)

print(f'Матрица A:\n{a}')
print(f'Собственные значения:\n{w}')
print(f'Собственные векторы:\n{v}')

Матрица A:
[[-1 -6]
 [ 2  6]]
Собственные значения:
[2. 3.]
Собственные векторы:
[[-0.89  0.83]
 [ 0.45 -0.55]]


__2.__ Дан оператор поворота на 180 градусов, задаваемый матрицей 

$$A=\begin{pmatrix}
-1 & 0\\ 
0 & -1
\end{pmatrix}.$$

Показать, что __любой__ вектор является для него собственным.


__Решение__

$$\begin{vmatrix}
-1-\lambda & 0\\ 
0 & -1-\lambda
\end{vmatrix} = 0,$$

$$(-1-\lambda)(-1-\lambda)-0\cdot0=0,$$

$$(-1-\lambda)^{2}=0,$$

Собственные значения: $$\lambda=-1.$$

$\textbf{A}x=\lambda x$:

$$\begin{pmatrix}
-1 & 0\\ 
0 & -1
\end{pmatrix}
\begin{pmatrix}
x_{1}\\ 
x_{2}
\end{pmatrix}=
\lambda
\begin{pmatrix}
x_{1}\\ 
x_{2}
\end{pmatrix}.
$$

Для собственного значения $\lambda=-1:$

$$\begin{cases}
-x_{1}+0 \cdot x_{2}= -x_{1}, \\
0 \cdot x_{1}-x_{2}= -x_{2}.
\end{cases}$$ 

$$\begin{cases}
- x_{1} + x_{1} = 0, \\
- x_{2} + x_{2} = 0;
\end{cases}$$ 

$$\begin{cases}
0 = 0, \\
0 = 0.
\end{cases}$$

Собственный вектор - это любой вектор с любыми координатами $x_{1}$ и $x_{2}.$

__3.__ Пусть линейный оператор задан матрицей

$$A=\begin{pmatrix}
1 & 1\\ 
-1 & 3
\end{pmatrix}.$$

Установить, является ли вектор $x=(1,1)$ собственным вектором этого линейного оператора.


__Решение__

Предположим, что вектор $x$ является собственным вектором заданного линейного оператора, тогда должно существовать некоторое вещественное число $\lambda$, при котором 

$$\begin{pmatrix}
1 & 1\\ 
-1 & 3
\end{pmatrix}
\begin{pmatrix}
1\\ 
-2 
\end{pmatrix}=
\lambda
\begin{pmatrix}
1\\ 
-2
\end{pmatrix}.
$$

Из этого будет следовать, что 

$$\begin{cases}
1-2=1\cdot\lambda \\ 
-1-7 = -2\cdot \lambda
\end{cases}
\Rightarrow
\begin{cases}
\lambda=-1\\ 
\lambda=3,5
\end{cases}.$$

Такая система не имеет смысла, следовательно, вектор $x=(1,-2)$ не является собственным вектором линейного оператора, заданного матрицей $A$.

__4.__ Пусть линейный оператор задан матрицей

$$A=\begin{pmatrix}
0 & 3 & 0\\ 
3 & 0 & 0\\
0 & 0 & 3
\end{pmatrix}.$$

Установить, является ли вектор $x=(3, -3, -4)$ собственным вектором этого линейного оператора.